In [104]:
import pandas as pd
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter

C:\Users\carol\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [100]:
class knn():
    def __init__(self):
        pass
    def train(self, X, y):
        self.X_train = X
        self.y_train = y
    def predict(self, X, k=1):
        dists = self.get_distances(X)
        n_test = dists.shape[0]
        y_pred = np.zeros(n_test)
        for i in range(n_test):
            k_closest_y = []
            labels = self.y_train[np.argsort(dists[i,:])].flatten()
            # find k nearest lables
            k_closest_y = labels[:k]
            c = Counter(k_closest_y)
            y_pred[i] = c.most_common(1)[0][0]
        return(y_pred)

    def get_distances(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        
        dot_prod = np.dot(X, self.X_train.T)
        
        sum_square_test = np.square(X).sum(axis = 1)
        sum_square_test = np.expand_dims(sum_square_test,axis=1)
 
        sum_square_train = np.square(self.X_train).sum(axis = 1)
        sum_square_train = np.expand_dims(sum_square_train,axis=0)
        
        dists = np.sqrt(-2 * dot_prod + sum_square_train + sum_square_test)

        return(dists) 

In [3]:
#get all names of files in the directory
path = "C:/Users/carol/DataSience/data/return-data/"
files = ([f for f in os.listdir(path) if f.endswith('.txt')])

In [4]:
#read files
df_list = []

for file in files:
    with open(path+file, 'r') as f:
        for line in f:
            data = json.loads(line)
            df_list.append(data)

In [113]:
#merge to one dataframe
dataframe = pd.DataFrame.from_dict(df_list)

def normalize_mean(input):
    return (input - input.mean()) / (input.max() - input.min())
def rescaling (input):
    return (input - input.min()) / (input.max() - input.min())
def normalize(input):
    return (input - input.mean()) / input.std()

In [91]:
X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(axis=1,columns=['basket','returnLabel','transactionId','zipCode']),dataframe['returnLabel'], test_size=0.30,random_state=101)


In [116]:

X_train = normalize(np.array(X_train))
X_test = normalize(np.array(X_test))
y_train = np.array(y_train)
y_test = np.array(y_test)
k = 5
classifier = knn()
classifier.train(X_train, y_train)
pred = np.array(classifier.predict(X_test, k))


print("Completed predicting the test data.")
#print(predictions)
print('Accuracy = {}'.format(np.sum(y_test == pred)/len(y_test)))



Completed predicting the test data.
Accuracy = 0.9620430107526882


In [114]:
#Start Exercise 2

# Drop column B as it is now encoded
#result = dataframe.apply(pd.value_counts(dataframe['basket'].values, sort=False))
#print(result)
count = dataframe['basket'].apply(pd.value_counts).fillna(0)
dataframe = dataframe.join(count)
X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(axis=1,columns=['basket','returnLabel','transactionId','zipCode']),dataframe['returnLabel'], test_size=0.30,random_state=101)
X_train = normalize(X_train)
X_test = normalize(X_test)
print(dataframe)
logmodel = RandomForestClassifier()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(predictions)
print(classification_report(y_test,predictions))
print("Accuracy:",accuracy_score(y_test, predictions))




                                           basket  returnLabel  totalAmount  \
0                                       [1, 5, 5]            0           30   
1                                 [2, 1, 1, 2, 2]            0          270   
2                        [5, 1, 5, 4, 4, 0, 2, 3]            0          648   
3                                    [2, 2, 5, 0]            0          332   
4                        [3, 3, 3, 0, 1, 3, 4, 4]            1          440   
5                                 [3, 3, 1, 1, 5]            0           50   
6                                       [5, 0, 1]            0           30   
7                     [4, 3, 3, 4, 0, 3, 5, 1, 4]            0          450   
8                                 [0, 1, 2, 4, 2]            0           50   
9                                    [4, 2, 3, 0]            0          148   
10                                [0, 2, 3, 2, 0]            0          340   
11                                      [2, 2, 0]   

[0 0 0 ... 0 0 0]
             precision    recall  f1-score   support

          0       0.98      0.99      0.99      9041
          1       0.47      0.30      0.37       259

avg / total       0.97      0.97      0.97      9300

Accuracy: 0.9710752688172043
